In [19]:
!pip3 install langchain langchain-openai langchain-community faiss-cpu tiktoken


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [20]:
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
import os

In [21]:
#os.environ['OPEN_API_KEY'] = 'sk-'

In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
loader = TextLoader("./data/sample1.txt")
documents = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=len)

In [26]:
docs = text_splitter.split_documents(documents)

In [30]:
len(docs)
docs[0:3]

[Document(metadata={'source': './data/sample1.txt'}, page_content='Metallica is an American heavy metal band. It was formed in Los Angeles in 1981 by vocalist and guitarist James Hetfield and drummer Lars Ulrich, and has been based in San Francisco for most of its career.[1][2] The band\'s fast tempos, instrumentals and aggressive musicianship made them one of the founding "big four" bands of thrash metal, alongside Megadeth, Anthrax and Slayer. Metallica\'s current lineup comprises founding members and primary songwriters Hetfield and Ulrich, longtime lead'),
 Document(metadata={'source': './data/sample1.txt'}, page_content='guitarist Kirk Hammett and bassist Robert Trujillo. Guitarist Dave Mustaine, who formed Megadeth after being fired from Metallica, and bassists Ron McGovney, Cliff Burton and Jason Newsted are former members of the band.'),
 Document(metadata={'source': './data/sample1.txt'}, page_content="Metallica first found commercial success with the release of its third albu

In [46]:
embedding = OpenAIEmbeddings()

In [48]:
library = FAISS.from_documents(docs, embedding)

In [49]:
query1 = "Who found Metallica?"

In [50]:
query_answer = library.similarity_search(query1)

In [53]:
query_answer[0].page_content

'Metallica was formed in Los Angeles in late 1981 when Danish drummer Lars Ulrich placed an advertisement in a Los Angeles newspaper, The Recycler, which read, "Drummer looking for other metal musicians to jam with Tygers of Pan Tang, Diamond Head and Iron Maiden."[9] Guitarists James Hetfield and Hugh Tanner of Leather Charm answered the advertisement. Although he had not formed a band, Ulrich asked Metal Blade Records founder Brian Slagel if he could record a song for the label\'s upcoming'

In [54]:
query_answer = library.similarity_search_with_score(query1)

In [55]:
query_answer

[(Document(id='47c7fae5-aa69-47a4-88cd-4cab8c1c1827', metadata={'source': './data/sample1.txt'}, page_content='Metallica was formed in Los Angeles in late 1981 when Danish drummer Lars Ulrich placed an advertisement in a Los Angeles newspaper, The Recycler, which read, "Drummer looking for other metal musicians to jam with Tygers of Pan Tang, Diamond Head and Iron Maiden."[9] Guitarists James Hetfield and Hugh Tanner of Leather Charm answered the advertisement. Although he had not formed a band, Ulrich asked Metal Blade Records founder Brian Slagel if he could record a song for the label\'s upcoming'),
  0.25353104),
 (Document(id='1a42f690-75a1-4618-a1f0-5b05655b8245', metadata={'source': './data/sample1.txt'}, page_content='Metallica founding members James Hetfield (top) and Lars Ulrich (bottom)'),
  0.2572756),
 (Document(id='7d711b01-a6a4-435f-a1d0-8d785b3dd7e6', metadata={'source': './data/sample1.txt'}, page_content='Metallica is an American heavy metal band. It was formed in Los

In [56]:
retriever = library.as_retriever()

In [57]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [62]:
query = "When was Master of Puppets recorded and when was it released"

In [63]:
results = qa.invoke(query)

In [64]:
results

{'query': 'When was Master of Puppets recorded and when was it released',
 'result': ' Master of Puppets was recorded from September to December 1985 and released in March 1986.'}

In [69]:
library.save_local("faiss_index_metallica")

In [71]:
metallica_saved = FAISS.load_local("faiss_index_metallica", embedding, allow_dangerous_deserialization=True)

In [72]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [75]:
results = qa.invoke("How many Grammy Awards did Metallica won?")

In [76]:
results

{'query': 'How many Grammy Awards did Metallica won?',
 'result': ' \n\nMetallica has won ten Grammy Awards from 26 nominations.'}